In [2]:
import os

from ladybug.sql import SQLiteResult
from ladybug import analysisperiod as ap

from eppy import *

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import pandas as pd

import numpy as np

from collections import OrderedDict

import datetime as dt

from IPython.display import display



In [3]:
root = os.getcwd()
root

'/Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local'

# SQL (Simulation) Analyis

In [4]:
def calc_metrics(simulated, measured):
    ts = simulated
    tm = measured
    tm_avg = np.mean(measured)
    n = len(tm)
    rmse = (100/tm_avg) * np.sqrt( (1/n) * np.sum((ts - tm)**2) )
    mbe = (100/tm_avg) * (np.sum(ts - tm)/n)
    return np.round(rmse,3), np.round(mbe,3)

In [5]:
def monthly_by_hour_from_sql(sql_dir_name):
    sqlFile = os.path.join(sql_dir_name, "eplusout.sql")
    sqld = SQLiteResult(sqlFile)

    elect = sqld.data_collections_by_output_name("Electricity:Facility")[-1]
    hot = sqld.data_collections_by_output_name("DistrictHeating:Facility")[-1]
    chill = sqld.data_collections_by_output_name("DistrictCooling:Facility")[-1]

    sim_month_by_hr = OrderedDict({
    "elect": elect.average_monthly_per_hour(),
    "hot": hot.average_monthly_per_hour(),
    "chill": chill.average_monthly_per_hour()
    })

    sim_monthly= OrderedDict({
    "elect": elect.total_monthly(),
    "hot": hot.total_monthly(),
    "chill": chill.total_monthly(),
    })

    return sim_month_by_hr, sim_monthly


## Load SQL Files

In [20]:
# new_dir_name = os.path.join(adjusted_model_dir, "03-02_02-Hourly-Meter")
sim_names = {
    0: "03-02_02-Hourly-Meter", # idf adjusted of the  "correct" baseline, group_models/Final_project_3-1-2022_2
    # 1: "03-02_03-Original_OSM", # original osm with incorrect material, orientation, geom
    # 2: "03-03_01-CHW_Time2_25", # change chilled water setpoint to 25 C
    # 3: "03-03_02-CHW_Time2_25_Summer", # try for the summer design day...
    # # 4: "03-03_03-CHW_Time2_26", # change chilled water setpoint to 26 C
    # # 5: "03-03_04-CHW_Time2_28",
    # 6: "03-03_05-CHW_Time2_29",
    # 7: "03-03_06-Raise_Chilled_Water_Temp_12",
    # 8: "03-03_07-new-baseline", # has 29 deg setpoint and 12 deg water...
    1: "03-03_08-new-baseline-reg", # has 24 deg setpoint and 6 deg water
    # 10: "03-03_09-heating",
    2: "03-03_09-heating_t2_17",
    3: "03-06_00-eh-6"
}

adjusted_model_dir = os.path.join(root, "eppy_adjusted_models")
sim_name_paths = {k: os.path.join(adjusted_model_dir, v) for k,v in sim_names.items()}

In [21]:
sim_data_hr = OrderedDict()
sim_data_month = OrderedDict()

for k, v in zip(sim_names.values(), sim_name_paths.values()):
    sim_data_hr[k] = monthly_by_hour_from_sql(v)[0]
    sim_data_month[k] = monthly_by_hour_from_sql(v)[1]

# Load and Convert Historical Data

In [22]:
# conversion factors
tonHr_kwHr = 3.5169 # 1 ton-hr = 3.517 kWh
kbtu_kwHr = 1/3.412 # 1 kbtu = 0.29 kwHr

# load data 
historical_data_path = os.path.join(root, "data/Thornton Utilities 2019.xlsx")

# electricity # kwh 
hist_elect_hourly = pd.read_excel(historical_data_path, sheet_name="Electricity hourly", usecols="A:E")

# chilled water  -> convert ton hours to kWh
hist_chw_hourly = pd.read_excel(historical_data_path, sheet_name="CHW hourly", usecols="A:E")
hist_chw_hourly["kWh"] = hist_chw_hourly["ton-hours"]*tonHr_kwHr

# hot water -> convert kbtu to kWh
hist_hw_hourly = pd.read_excel(historical_data_path, sheet_name="HW hourly", usecols="A:E")
hist_hw_hourly["kWh"] = hist_hw_hourly["kBtu"]*kbtu_kwHr 

In [23]:
# month by hour
hist_month_by_hr = OrderedDict({
    "elect" : hist_elect_hourly.groupby(["Month", "Hour"])["kWh"].mean(),
    "hot": hist_hw_hourly.groupby(["Month", "Hour"])["kWh"].mean(),
    "chill": hist_chw_hourly.groupby(["Month", "Hour"])["kWh"].mean(),
})

# monthly 
hist_monthly = OrderedDict({
   "elect": hist_elect_hourly.groupby("Month")["kWh"].sum(),
   "hot":  hist_hw_hourly.groupby("Month")["kWh"].sum(),
   "chill":  hist_chw_hourly.groupby("Month")["kWh"].sum()
})

# Comparison of Hourly Trends

In [24]:
hours = [dt.datetime(2019, 1, 1, i ).strftime("%H:%M") for i in range(24)]

In [25]:
def create_analysis_period(num):
    return ap.AnalysisPeriod(st_month=num, end_month=num)

In [26]:
def plot_datasets_monthly_by_hour(month_num, month_name):

    month = create_analysis_period(month_num)
    blueShades = px.colors.qualitative.Vivid[:len(sim_data_hr.keys())]

    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=("Electricity", "Hot Water", "Chilled Water"))

    # historical/ measured data 
    for (ix, value) in  enumerate(hist_month_by_hr.values()):
        showLegend = True
        if ix > 0:
            showLegend = False
        fig.add_trace(go.Scatter(  
            x=hours,
            y=value[month.st_month] , 
            mode='lines',
            name="Historical",
            legendgroup="Months",
            line=dict(color='red'),
            showlegend=showLegend),
            row=1, col=ix+1)

    # simulated datasets 
    for (name, dataset), blueShade in zip(sim_data_hr.items(), blueShades):
        for (ix, sim), hist in zip(enumerate(dataset.values()),hist_month_by_hr.values()) :
            showLegend = True
            if ix > 0:
                showLegend = False
            fig.add_trace(go.Scatter( 
                x=hours,
                y=sim.filter_by_analysis_period(month).values , 
                mode='lines+markers',
                name=name,
                legendgroup="Months",
                line=dict(color=blueShade),
                showlegend=showLegend),
                row=1, col=ix+1)

    fig.update_layout(title=f"{month_name} Average Hourly Energy Usage",
        yaxis_title="Energy (kWh)",)
   
   # calculate metrics 
    metrics = {}
    for name, dataset in sim_data_hr.items():
        type_dict = {}
        for (k, sim), hist in zip(dataset.items(), hist_month_by_hr.values()):
            type_dict[k] = calc_metrics(sim.filter_by_analysis_period(month).values, hist[month.st_month])
        metrics[name] = type_dict
    metrics = pd.DataFrame(metrics)
    display(metrics)

    fig.show()

In [27]:
plot_datasets_monthly_by_hour(1, "Jan")

,03-02_02-Hourly-Meter,03-03_08-new-baseline-reg,03-03_09-heating_t2_17,03-06_00-eh-6
elect,"(22.85, 7.58)","(46.977, -44.24)","(47.303, -44.574)","(40.458, -37.025)"
hot,"(105.128, -79.158)","(95.625, -65.408)","(118.922, -85.278)","(105.887, -70.523)"
chill,"(416.617, 259.266)","(88.273, 30.007)","(59.889, -37.366)","(110.808, 35.68)"


In [28]:
plot_datasets_monthly_by_hour(6, "June")

Updated end_day from 31 to 30


,03-02_02-Hourly-Meter,03-03_08-new-baseline-reg,03-03_09-heating_t2_17,03-06_00-eh-6
elect,"(37.717, 22.334)","(42.488, -36.965)","(41.948, -36.631)","(35.767, -27.369)"
hot,"(99.739, -91.316)","(85.916, -81.088)","(109.801, -96.407)","(96.855, -89.328)"
chill,"(124.112, 91.077)","(72.709, 37.885)","(63.097, -23.339)","(80.306, 37.637)"


# Comparison of Annual Trends 

In [29]:
months = [dt.datetime(2019, i+1, 1).strftime("%b") for i in range(12)]

In [30]:
def plot_datasets_monthly():
    blueShades = px.colors.qualitative.Vivid[:len(sim_data_hr.keys())]

    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=("Electricity", "Hot Water", "Chilled Water"))

    # historical/ measured data 
    for (ix, value) in  enumerate(hist_monthly.values()):
        showLegend = True
        if ix > 0:
            showLegend = False
        fig.add_trace(go.Scatter(  
            x=months,
            y=value, 
            mode='lines',
            name="Historical",
            legendgroup="Months",
            line=dict(color='red'),
            showlegend=showLegend),
            row=1, col=ix+1)

    # simulated datasets 
    for (name, dataset), blueShade in zip(sim_data_month.items(), blueShades):
        for (ix, sim), hist in zip(enumerate(dataset.values()),hist_monthly.values()) :
            showLegend = True
            if ix > 0:
                showLegend = False
            fig.add_trace(go.Scatter( 
                x=months,
                y=sim.values , 
                mode='lines+markers',
                name=name,
                legendgroup="Months",
                line=dict(color=blueShade),
                showlegend=showLegend),
                row=1, col=ix+1)

    fig.update_layout(title=f"Monthly Total Energy Usage",
        yaxis_title="Energy (kWh)",)
   
   # calculate metrics 
    metrics = {}
    for name, dataset in sim_data_month.items():
        type_dict = {}
        for (k, sim), hist in zip(dataset.items(), hist_monthly.values()):
            type_dict[k] = calc_metrics(sim.values, hist)
        metrics[name] = type_dict
    metrics = pd.DataFrame(metrics)
    display(metrics)

    fig.show()

In [31]:
plot_datasets_monthly()

,03-02_02-Hourly-Meter,03-03_08-new-baseline-reg,03-03_09-heating_t2_17,03-06_00-eh-6
elect,"(18.173, 14.711)","(41.964, -40.69)","(42.079, -40.741)","(34.158, -32.478)"
hot,"(94.587, -83.009)","(79.961, -69.692)","(100.898, -89.117)","(84.888, -74.878)"
chill,"(144.354, 128.225)","(66.133, 47.678)","(29.387, -22.064)","(68.056, 51.115)"


In [32]:
hist_monthly["hot"]

Month
1      9563.779652
2     11205.168864
3     10021.888756
4      5739.921388
5      8015.771756
6      2812.879287
7      1998.993427
8      1290.168084
9      1966.378451
10     4721.300882
11     7453.441033
12    10536.379511
Name: kWh, dtype: float64

In [33]:
# {'elect': 3.100677715042496, 'hot': 1.8389211816597943, 'chill': 9.354337655572776}
    